<a href="https://colab.research.google.com/github/mitali72/Generating-aligned-transcripts/blob/main/Vakyansh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install webrtcvad wave pydub resemblyzer hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 66 kB 1.6 MB/s 
     |████████████████████████████████| 15.7 MB 4.4 MB/s 
     |████████████████████████████████| 5.2 MB 45.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 7.3 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72377 sha256=7674766a4757f4c2774a15cfd7ecad27ff906bad8fae2509b54c0adcc555e511
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
  Created wheel for wave: filename=Wave-0.0.2-py3-none-any.whl size=1240 sha256=3dc41d47bb56d1799c058252a6bf9e81d46b2906638d9a91c295aaf9f756d074
  Stored in directory: /root/.cache/pip/wheels/25/e8/fe/458c7dac00c6abedad6380b9d0ef1a5cbc7c21807df1d30915
  Created wheel for 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load sample data
!unzip /content/drive/MyDrive/BTP/NewsOnAir.zip -d /content/NewsOnAir

Archive:  /content/drive/MyDrive/BTP/NewsOnAir.zip
   creating: /content/NewsOnAir/Gujarati/
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1320-1330-202262213512.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1320-1330-202274133759.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-20225481235.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-2022819250.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-20224148830.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1320-1330-202276134830.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1950-2000-20228520538.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-202282184951.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-202282781351.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1950-2000-20228921348

In [ ]:
# display(Audio("https://newsonair.gov.in/writereaddata/Bulletins_Audio/NSD/2022/Aug/NSD-Gujarati-Gujarati-0745-0755-202281981355.mp3"))
from IPython.display import Audio
from IPython.display import display
from pydub import AudioSegment
import os

# Convert all mp3 files to wav with 16kHz sampling rate
mp3_folder = "/content/NewsOnAir/"
wav_folder = "/content/NewsOnAirWav"
!mkdir "$wav_folder"
for root, dirs, files in os.walk(mp3_folder):
  for audio_file in files:
    audio_path = os.path.join(root,audio_file)
    sound = AudioSegment.from_mp3(audio_path)
    sound = sound.set_frame_rate(16000)
    sound = sound.set_channels(1)
    wav_dir = os.path.join("/content/NewsOnAirWav/",audio_path.split("/")[3])
    if not os.path.exists(wav_dir):
      os.mkdir(wav_dir)

    sound.export(os.path.join(wav_dir,audio_path.split("/")[-1][:-3]+"wav"), format="wav")


# **VAD**

In [ ]:
import webrtcvad
import contextlib
import wave
import collections
import sys
import os

def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        # sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                # sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                # sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    # if triggered:
        # sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    # sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

In [ ]:
from pathlib import Path

# Save chunks after VAD
def save_vad(segments, sample_rate,wav_file = ""):
  
  # chunk_snr = []
  audio_name = Path(wav_file).stem
  chunk_path = os.path.join('/content/NewsOnAirVAD',wav_file.split("/")[3],audio_name)
  Path(chunk_path).mkdir(parents = True,exist_ok=True)
  for i, segment in enumerate(segments):
    write_wave(os.path.join(chunk_path,'chunk-%002d.wav' % (i,)), segment, sample_rate)


In [ ]:

for root, dirs, files in os.walk(wav_folder):
  for audio_file in files:
    audio_path = os.path.join(root,audio_file)
    audio, sample_rate = read_wave(audio_path)
    vad = webrtcvad.Vad(mode = 2)
    frames = frame_generator(30, audio, sample_rate)
    frames = list(frames)
    # Voice chunks
    segments = vad_collector(sample_rate, 30, 300, vad, frames)
    save_vad(segments,sample_rate,audio_path)
    # snr_passed,chunk_index = check_snr(segments)
    # segment_names.append(os.path.join())



In [ ]:
# Join chunks after VAD
output = []
vad_path = "/content/NewsOnAirVAD/Gujarati/NSD-Gujarati-Gujarati-1320-1330-202282814554"
for chunk_name in sorted(os.listdir(vad_path)):
  chunk_path = os.path.join(vad_path,chunk_name)
  if len(output)>0:
    output[0] += AudioSegment.from_file(chunk_path, format="wav", sample_rate=16000)
  else:
    output.append(AudioSegment.from_file(chunk_path, format="wav", sample_rate=16000))

output[0].export("only_speech_guj554_webrtc.wav",format = "wav")


# **SNR thresholding & Speaker Clustering** (Resemblyzer)

In [ ]:
import numpy as np

def wada_snr(wav):
    # Direct blind estimation of the SNR of a speech signal.
    #
    # Paper on WADA SNR:
    #   http://www.cs.cmu.edu/~robust/Papers/KimSternIS08.pdf
    #
    # This function was adapted from this matlab code:
    #   https://labrosa.ee.columbia.edu/projects/snreval/#9

    # init
    eps = 1e-10
    # next 2 lines define a fancy curve derived from a gamma distribution -- see paper
    db_vals = np.arange(-20, 101)
    g_vals = np.array([0.40974774, 0.40986926, 0.40998566, 0.40969089, 0.40986186, 0.40999006, 0.41027138, 0.41052627, 0.41101024, 0.41143264, 0.41231718, 0.41337272, 0.41526426, 0.4178192 , 0.42077252, 0.42452799, 0.42918886, 0.43510373, 0.44234195, 0.45161485, 0.46221153, 0.47491647, 0.48883809, 0.50509236, 0.52353709, 0.54372088, 0.56532427, 0.58847532, 0.61346212, 0.63954496, 0.66750818, 0.69583724, 0.72454762, 0.75414799, 0.78323148, 0.81240985, 0.84219775, 0.87166406, 0.90030504, 0.92880418, 0.95655449, 0.9835349 , 1.01047155, 1.0362095 , 1.06136425, 1.08579312, 1.1094819 , 1.13277995, 1.15472826, 1.17627308, 1.19703503, 1.21671694, 1.23535898, 1.25364313, 1.27103891, 1.28718029, 1.30302865, 1.31839527, 1.33294817, 1.34700935, 1.3605727 , 1.37345513, 1.38577122, 1.39733504, 1.40856397, 1.41959619, 1.42983624, 1.43958467, 1.44902176, 1.45804831, 1.46669568, 1.47486938, 1.48269965, 1.49034339, 1.49748214, 1.50435106, 1.51076426, 1.51698915, 1.5229097 , 1.528578  , 1.53389835, 1.5391211 , 1.5439065 , 1.54858517, 1.55310776, 1.55744391, 1.56164927, 1.56566348, 1.56938671, 1.57307767, 1.57654764, 1.57980083, 1.58304129, 1.58602496, 1.58880681, 1.59162477, 1.5941969 , 1.59693155, 1.599446  , 1.60185011, 1.60408668, 1.60627134, 1.60826199, 1.61004547, 1.61192472, 1.61369656, 1.61534074, 1.61688905, 1.61838916, 1.61985374, 1.62135878, 1.62268119, 1.62390423, 1.62513143, 1.62632463, 1.6274027 , 1.62842767, 1.62945532, 1.6303307 , 1.63128026, 1.63204102])

    # peak normalize, get magnitude, clip lower bound
    wav = np.array(wav)
    wav = wav / abs(wav).max()
    abs_wav = abs(wav)
    abs_wav[abs_wav < eps] = eps

    # calcuate statistics
    # E[|z|]
    v1 = max(eps, abs_wav.mean())
    # E[log|z|]
    v2 = np.log(abs_wav).mean()
    # log(E[|z|]) - E[log(|z|)]
    v3 = np.log(v1) - v2

    # table interpolation
    wav_snr_idx = None
    if any(g_vals < v3):
        wav_snr_idx = np.where(g_vals < v3)[0].max()
    # handle edge cases or interpolate
    if wav_snr_idx is None:
        wav_snr = db_vals[0]
    elif wav_snr_idx == len(db_vals) - 1:
        wav_snr = db_vals[-1]
    else:
        wav_snr = db_vals[wav_snr_idx] + \
            (v3-g_vals[wav_snr_idx]) / (g_vals[wav_snr_idx+1] - \
            g_vals[wav_snr_idx]) * (db_vals[wav_snr_idx+1] - db_vals[wav_snr_idx])

    # Calculate SNR
    dEng = sum(wav**2)
    dFactor = 10**(wav_snr / 10)
    dNoiseEng = dEng / (1 + dFactor) # Noise energy
    dSigEng = dEng * dFactor / (1 + dFactor) # Signal energy
    snr = 10 * np.log10(dSigEng / dNoiseEng)

    return wav_snr

In [ ]:
# Check SNR thresholds
def check_snr(segments):
  """
  segments: generator
  """
  snr_passed = []
  chunk_index = []
  for i,segment in enumerate(segments):
    chunk_snr = wada_snr(segment)
    if chunk_snr>=20 or chunk_snr<=60:
      snr_passed.append(segment)
      chunk_index.append(i)

  return snr_passed,chunk_index


In [ ]:
from resemblyzer import VoiceEncoder, preprocess_wav
from pathlib import Path
import numpy as np
from scipy.io import wavfile
import csv


encoder = VoiceEncoder()
chunk_fields = ["audio_name", "chunk_id", "snr", "len"]
vad_folder = "/content/NewsOnAirVAD/"

f = open('chunk_info.csv', 'w')
writer = csv.DictWriter(f, fieldnames = chunk_fields)
writer.writeheader()
embedv = []
for root, dirs, files in os.walk(vad_folder):
  chunk_info = {}
  
  for audio_file in sorted(files):
    
    audio_path = os.path.join(root,audio_file)
    _,wav = wavfile.read(audio_path)
    chunk_snr = wada_snr(wav)
    embed = encoder.embed_utterance(preprocess_wav(audio_path))

    chunk_info["audio_name"] = audio_path.split("/")[-2]
    chunk_info["chunk_id"] = Path(audio_file).stem[-2:]
    chunk_info["snr"] = chunk_snr
    chunk_info["len"] = len(wav)/16000
    embedv.append(embed)
 
    writer.writerow(chunk_info)

f.close()

Loaded the voice encoder model on cpu in 0.05 seconds.


In [ ]:
import pandas as pd
from ast import literal_eval
import json


df = pd.read_csv('chunk_info.csv')
df["embed"] = embedv
snr_passed = df.loc[(df["snr"]>=20) & (df["snr"]<=60)]

In [ ]:
import hdbscan

# Noisy samples given -1
clusterer = hdbscan.HDBSCAN(min_cluster_size=4, allow_single_cluster=True, min_samples=2
                            )
cluster_labels = clusterer.fit_predict(np.stack(snr_passed["embed"].values))
snr_passed["cluster_label"] = cluster_labels
output = snr_passed.sort_values(by=['cluster_label','snr'], ascending=[True, False])
output.to_csv("output.csv", columns = ["audio_name", "chunk_id", "snr","cluster_label"], index = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
